In [1]:
import pandas as pd
import os
import requests

In [2]:
os.listdir()

['.git',
 '.gitignore',
 '.ipynb_checkpoints',
 'census_api_key.txt',
 'disaster_vulnerability_score_819.xlsx',
 'fema_funding_by_state',
 'Kentucky NFIP Analysis.ipynb',
 'Kentucky_Story_Data_Analysis.ipynb',
 'README.md',
 'top_hmgp_funding',
 'top_public_assistance_funding',
 'top_total_funding',
 'Vulnerability Analysis.ipynb']

In [3]:
df = pd.read_excel("disaster_vulnerability_score_819.xlsx")
print(df.shape)
df.head()

(9275, 65)


,disasternumber,ihprogramdeclared,iaprogramdeclared,paprogramdeclared,hmprogramdeclared,state,declarationdate,fydeclared,incidenttype,title,...,unemployment_rate_us,elderly_score,youth_score,race_score,unemploy_score,med_hh_inc_score,hh_snap_score,hh_renter_score,total_score,region
0,4301,False,False,True,True,CA,2017-02-14T18:15:00,2017,Severe Storm(s),"SEVERE WINTER STORMS, FLOODING, AND MUDSLIDES",...,0.065527,0.0,1.0,1.0,1.0,1,1.0,1.0,6.0,West
1,4077,False,False,True,True,OH,2012-08-20T16:10:00,2012,Severe Storm(s),SEVERE STORMS AND STRAIGHT-LINE WINDS,...,0.092866,1.0,0.0,0.0,1.0,1,1.0,0.0,4.0,Midwest
2,4317,False,False,True,True,MO,2017-06-02T16:56:00,2017,Flood,"SEVERE STORMS, TORNADOES, STRAIGHT-LINE WINDS ...",...,0.065527,1.0,1.0,0.0,1.0,1,1.0,1.0,6.0,Midwest
3,1980,True,False,False,True,MO,2011-05-09T21:00:00,2011,Severe Storm(s),"SEVERE STORMS, TORNADOES, AND FLOODING",...,0.086840,1.0,0.0,0.0,0.0,1,1.0,1.0,4.0,Midwest
4,1915,False,False,True,True,SD,2010-05-13T16:29:00,2010,Flood,FLOODING,...,0.079228,0.0,0.0,0.0,0.0,1,0.0,1.0,2.0,Midwest


In [4]:
key = open('census_api_key.txt','r').read()

In [5]:
def state_year_SNAP_prop(state, year, key = key):
    if int(year) > 2017: year = "2017" # ACS data isn't available beyond 2017
    snap_stat = "B22001_002E"
    base_url = "https://api.census.gov/data/%s/acs/acs5?" % year
    if year == "2009": base_url = "https://api.census.gov/data/2009/acs5?" # API url is different for 2009
    end_url = "get=NAME,%s&for=state:%s&key=%s" % (snap_stat, state, key)
    response = requests.get(base_url+end_url)
    snap_recipients = response.json()[1][1]
    
    no_snap_stat = "B22001_005E"
    end_url = "get=NAME,%s&for=state:%s&key=%s" % (no_snap_stat, state, key)
    response = requests.get(base_url+end_url)
    no_snap_recipients = response.json()[1][1]
    return int(snap_recipients) / (int(snap_recipients)+int(no_snap_recipients))

In [6]:
df['calendar_year'] = df['incidentbegindate'].apply(lambda x: x[:4])
df['calendar_year'].value_counts().sort_index()

2009    1171
2010    1114
2011    1762
2012     630
2013     691
2014     470
2015    1098
2016     646
2017    1051
2018     642
Name: calendar_year, dtype: int64

In [7]:
table = df.drop_duplicates(subset = ['calendar_year','state'])[['calendar_year','state','state_fips']].copy()
states_of_interest = ['KY','OK','WV','SD','NE','KS','AR','WY','ND','LA','MN','ID']
table = table[table['state'].isin(states_of_interest)]
for state in states_of_interest: # checking to make sure I didn't miss any
    if state not in table['state'].unique(): print(state)
table['county_eq'] = table.apply(lambda row: 
    df[(df['state_fips']==row['state_fips'])&(df['calendar_year']==row['calendar_year'])]['declaredcountyarea'].nunique(),
                                 axis = 1)
table['state_fips'] = table['state_fips'].apply(lambda x: str(x) if x >= 10 else '0'+str(x))
table['state_year_snap'] = table.apply(lambda row:
            state_year_SNAP_prop(state = row['state_fips'], year = row['calendar_year']), axis = 1)
table.head()

,calendar_year,state,state_fips,county_eq,state_year_snap
4,2010,SD,46,55,0.085223
8,2009,NE,31,67,0.070423
9,2013,KS,20,56,0.093690
12,2009,KS,20,79,0.068420
14,2010,NE,31,72,0.075127


In [8]:
table['higher_than_state_snap_counties'] = table.apply(lambda row:
    len(df[(df['state_fips']==int(row['state_fips']))&(df['calendar_year']==row['calendar_year'])&(df['pct_hh_snap']>row['state_year_snap'])].drop_duplicates(
        subset = ['declaredcountyarea'])), axis = 1)
table['prop_counties_higher_snap_than_state'] = table['higher_than_state_snap_counties'] / table['county_eq']
table.head()

,calendar_year,state,state_fips,county_eq,state_year_snap,higher_than_state_snap_counties,prop_counties_higher_snap_than_state
4,2010,SD,46,55,0.085223,17,0.309091
8,2009,NE,31,67,0.070423,21,0.313433
9,2013,KS,20,56,0.093690,23,0.410714
12,2009,KS,20,79,0.068420,35,0.443038
14,2010,NE,31,72,0.075127,16,0.222222


In [12]:
#table.to_csv("state_snap_info.csv", index = False)

In [10]:
ky_table = table[table['state']=="KY"].sort_values(by = "calendar_year")
ky_table

,calendar_year,state,state_fips,county_eq,state_year_snap,higher_than_state_snap_counties,prop_counties_higher_snap_than_state
33,2009,KY,21,108,0.132344,64,0.592593
211,2010,KY,21,85,0.139813,55,0.647059
254,2011,KY,21,81,0.148587,45,0.555556
391,2012,KY,21,23,0.160118,17,0.739130
5928,2014,KY,21,4,0.172815,4,1.000000
285,2015,KY,21,84,0.173225,58,0.690476
498,2016,KY,21,20,0.168937,11,0.550000
247,2018,KY,21,55,0.160726,37,0.672727


In [13]:
#ky_table.to_csv("ky_snap_info.csv", index = False)